# Purity Estimation Experiment

This estimates the purity of a quantum state using the Unitary-2 design identity as described in arXiv:1812.02624

This generates the circuits for appending a state preparation circuit with the tensor product of 1-qubit Cliffords before measurement and analysis consists of averaging over products of outcome probabilities to estimate $Tr[\rho^2]$ as
$$Tr[\rho^2 = 2^N \sum_{i,j}\sum_{C} (-2)^{w_{i,j}} p_i(C) p_j(C)$$

where $w_{i,j}$ is the Hamming weight distance between two bitstrings i and j, and $p_i(C)$ is the measurement outcome probability $p_i(C) = \langle i | C\rho C^\dagger | i \rangle$, where $C = \bigotimes_{i=1}^N C_i$ is a tensor product of 1-qubit Cliffords.

Note that whilte there are 24 1-qubit Cliffords, however there are only 6 equivalence classes of cliffords that need to be sampled from since the 4 Cliffords in each class generate the same measurement outcomes. This means there are a total of $6^N$ measurement circuits to sample from for an $N$-qubit measurement.

For our experiment if `num_samples = None` all $6^N$ measurement circuits are generated and run. If `num_samples` < $6^N$ we sample without replacement that number of measurement circuits to execute.

In [ ]:
import qiskit
import qiskit.quantum_info as qi
from qiskit import IBMQ
from qiskit.providers.aer import AerSimulator
from qiskit.test import mock
from qiskit.circuit.library import QuantumVolume

# Import purity estimation experiment from qiskit-experiments dev branch
from qiskit_experiments import PurityEstimation

# Example: Purity of QV circuit

We now demonstrate running the experiment for to compute the purity of a Quantum Volume circuit.

In [2]:
# Load provider and get backend
IBMQ.load_account()
provider = IBMQ.get_provider(group='deployed')
backend = provider.get_backend('ibmq_mumbai')

# Create simulator backend of device
sim = AerSimulator.from_backend(backend)

In [3]:
# Make QV circuit
# Run only depth-1 because otherwise fidelity is bad
prep_circ = QuantumVolume(5, 1, seed=101)

# Transpile with truncated coupling map
coupling_map = [[0, 1], [1, 0], [1, 2], [1, 4], [2, 1], [2, 3], [3, 2]]
prep_circ = qiskit.transpile(
    prep_circ, backend, coupling_map=coupling_map, optimization_level=3)

## Simulation 

### Direct Density Matrix Calculation

First we directly simulate the noisy density matrix so we can compute the target purity our protocol should be estimating. This effectively simulates gate errors only, no measurement errors are included.

In [4]:
# Directly simulate noisy circuit to get the "true" noisy target density matrix
qc = prep_circ.copy()
qc.save_density_matrix()
result = sim.run(qc).result()
rho_target = result.data(0)['density_matrix']

# Calculate purity
purity_target = qi.DensityMatrix(rho_target).purity().real
print('Target Purity = {:.4f}'.format(purity_target))

Target Purity = 0.9261


### Purity Experiment Estimation

Now we run a simulation of the full experimetn drawing 500 samples from the full 6^5 = 7776 possible random Cliffords. This includes simulation of readout error (we probably need to look into applying readout error mitigation)

In [5]:
# Experiment parameters
num_samples = 200
seed = 1234
shots = 1000

In [8]:
exp_sim = PurityEstimation(prep_circ, num_samples=num_samples, seed=seed)
expdata_sim = exp_sim.run(sim, shots=shots)
purity_exp_sim = expdata_sim.analysis_result(-1)['purity']
print('Purity (Simulated) = {:.4f}'.format(purity_exp_sim))

Purity (Simulated) = 0.7436


Run again disabling readout error (ie test protocol with only gate error)

In [9]:
# Create simulator with gate-only device noise model (no measuerment readout error)
sim_noro = AerSimulator.from_backend(backend, readout_error=False)

exp_sim_noro = PurityEstimation(prep_circ, num_samples=num_samples, seed=seed)
expdata_sim_noro = exp_sim.run(sim_noro, shots=shots)
purity_exp_sim_noro = expdata_sim_noro.analysis_result(-1)['purity']
print('Purity (Simulated, No readout error) = {:.4f}'.format(purity_exp_sim_noro))

Purity (Simulated, No readout error) = 0.7644


## Actual Device

Now lets run on the actual device. Note the experiment class API isn't really suitable at the moment for running on devices (since it blocks until the job is finished). It is better to run manually as follows.

In [6]:
# Generate circuits
exp_dev = PurityEstimation(prep_circ, num_samples=num_samples, seed=seed)
circuits = exp_dev.transpiled_circuits(backend)
job = backend.run(circuits, shots=shots)
print(job.job_id())

609183d2e60483510f734632


In [11]:
# Monitor job
job.status()

<JobStatus.DONE: 'job has successfully run'>

In [12]:
from qiskit_experiments import ExperimentData

# Construct experiment data container
expdata_dev = ExperimentData(exp_dev)
expdata_dev.add_data(job)

# Analyze experiment data
exp_dev.analysis().run(expdata_dev)

# Get purity
purity_exp_dev = expdata_dev.analysis_result(-1)['purity']
print('Purity (Experiment) = {:.4f}'.format(purity_exp_dev))

Purity (Experiment) = 0.7019
